In [1]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os 

In [2]:
csv_filename = 'feret_without_generations.csv' 
df = pd.read_csv(csv_filename)

In [3]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [4]:
test_df

,filename,gender,race,is_generated
647,00707_941205_fa_converted_final.png,0,0,False
607,00663_941121_fa_converted_final.png,1,0,False
63,00099_931230_fa_a_converted_final.png,0,2,False
319,00364_940422_fa_converted_final.png,0,0,False
101,00138_931230_fa_converted_final.png,0,0,False
...,...,...,...,...
110,00147_931230_fa_converted_final.png,1,2,False
82,00118_931230_fa_a_converted_final.png,0,0,False
51,00086_931230_fa_converted_final.png,0,0,False
428,00480_940519_fa_converted_final.png,1,0,False


In [5]:
generated_df = pd.read_csv("feret_generated.csv")
train_df = pd.concat([train_df, generated_df], ignore_index=True)

In [6]:
root_path = "/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colored/"
generation_path = "/home/mahdi/Projects/FairnessLens/Gateway/results/feret/"
train_df['image_path'] = train_df['filename'].apply(lambda x: os.path.join(root_path if "final" in x else generation_path, x))
test_df['image_path'] = test_df['filename'].apply(lambda x: os.path.join(root_path, x))

In [7]:
train_image_paths = train_df['image_path'].tolist()
test_image_paths = test_df['image_path'].tolist()
train_image_paths

['/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colored/00738_941201_fa_converted_final.png',
 '/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colored/00500_940519_fa_converted_final.png',
 '/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colored/00733_941201_fa_converted_final.png',
 '/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colored/00053_931230_fa_converted_final.png',
 '/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colored/00530_940519_fa_converted_final.png',
 '/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colored/00504_940519_fa_converted_final.png',
 '/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colored/00700_941121_fa_converted_final.png',
 '/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colored/00097_931230_fa_converted_final.png',
 '/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colored/00194_940128_fa_converted_final.png',
 '/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colore

In [2]:
# Read and resize images
def preprocess_image(file_path, target_size=(512, 768)):
    img = cv2.imread(file_path) 
    img = cv2.resize(img, (512,512))
    x, y = 85, 0
    crop_img = img[y:y+512, x:x+342]
    resized_img = cv2.resize(crop_img, target_size)
    final_img = resized_img.astype("float") / 255.0  # Normalize pixel values
    cv2.imshow("test", resized_img)
    cv2.waitKey(0)
    return final_img

In [ ]:
preprocess_image("/home/mahdi/Projects/FairnessLens/FERET/colorferet/dvd1/colored/00006_931230_fa_converted_final.png")

In [9]:
# Apply preprocessing to the entire dataset
X_train = [preprocess_image(file_path) for file_path in train_image_paths]
X_test = [preprocess_image(file_path) for file_path in test_image_paths]

X_train = np.array(X_train)
X_test = np.array(X_test)

In [10]:
X_train.shape

(615, 384, 256, 3)

In [11]:
train_gender_labels = train_df['gender'].tolist()
test_gender_labels = test_df['gender'].tolist()
train_race_labels = train_df['race'].tolist()
test_race_labels = test_df['race'].tolist()

In [12]:
from sklearn.preprocessing import LabelEncoder

# Create label encoders for gender and race
gender_encoder = LabelEncoder()
race_encoder = LabelEncoder()

# Apply label encoding to gender
y_train_gender_encoded = gender_encoder.fit_transform(train_gender_labels)
y_test_gender_encoded = gender_encoder.transform(test_gender_labels)

# Apply label encoding to race
y_train_race_encoded = race_encoder.fit_transform(train_race_labels)
y_test_race_encoded = race_encoder.transform(test_race_labels)


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

2024-01-13 12:45:51.215442: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-13 12:45:51.430729: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-13 12:45:51.430788: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-13 12:45:51.445854: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-13 12:45:51.479587: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-13 12:45:51.480111: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [14]:
# Assuming RGB images with dimensions 100x150 pixels
input_shape = (384,256, 3)
num_classes_gender = 2 

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes_gender, activation='softmax'))  # For gender classification

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2024-01-13 12:45:53.281704: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 397426688 exceeds 10% of free system memory.
2024-01-13 12:45:53.370714: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 397426688 exceeds 10% of free system memory.
2024-01-13 12:45:53.408664: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 397426688 exceeds 10% of free system memory.


In [15]:
# Example training code
model.fit(X_train, y_train_gender_encoded, epochs=5, batch_size=5, validation_data=(X_test, y_test_gender_encoded))

Epoch 1/5


2024-01-13 12:45:56.269743: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 725483520 exceeds 10% of free system memory.
2024-01-13 12:45:56.847308: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 397426688 exceeds 10% of free system memory.


123/123 [==============================] - 108s 873ms/step - loss: 3.7241 - accuracy: 0.6748 - val_loss: 0.6636 - val_accuracy: 0.6912
Epoch 2/5
123/123 [==============================] - 92s 746ms/step - loss: 0.3981 - accuracy: 0.8537 - val_loss: 0.5239 - val_accuracy: 0.7647
Epoch 3/5
123/123 [==============================] - 88s 716ms/step - loss: 0.1641 - accuracy: 0.9561 - val_loss: 0.5248 - val_accuracy: 0.7426
Epoch 4/5
123/123 [==============================] - 86s 700ms/step - loss: 0.0547 - accuracy: 0.9902 - val_loss: 0.4682 - val_accuracy: 0.7647
Epoch 5/5
123/123 [==============================] - 87s 704ms/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.4777 - val_accuracy: 0.8015


In [23]:
loss, accuracy = model.evaluate(X_test, y_test_gender_encoded)
print(f"Test Accuracy: {accuracy}")

5/5 [==============================] - 1s 141ms/step - loss: 0.5771 - accuracy: 0.8603
Test Accuracy: 0.8602941036224365


In [24]:
input_shape = (384,256, 3)
num_classes_race = 5 

race_model = Sequential()
race_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
race_model.add(MaxPooling2D((2, 2)))
race_model.add(Flatten())
race_model.add(Dense(128, activation='relu'))
race_model.add(Dense(num_classes_race, activation='softmax'))

race_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [25]:
race_model.fit(X_train, y_train_race_encoded, epochs=5, batch_size=5, validation_data=(X_test, y_test_race_encoded))

Epoch 1/5
123/123 [==============================] - 92s 746ms/step - loss: 12.7348 - accuracy: 0.4081 - val_loss: 1.4176 - val_accuracy: 0.5441
Epoch 2/5
123/123 [==============================] - 92s 746ms/step - loss: 1.0992 - accuracy: 0.5951 - val_loss: 1.1637 - val_accuracy: 0.6397
Epoch 3/5
123/123 [==============================] - 92s 746ms/step - loss: 0.6174 - accuracy: 0.8000 - val_loss: 1.2359 - val_accuracy: 0.6176
Epoch 4/5
123/123 [==============================] - 92s 749ms/step - loss: 0.2786 - accuracy: 0.9431 - val_loss: 1.3919 - val_accuracy: 0.5000
Epoch 5/5
123/123 [==============================] - 92s 749ms/step - loss: 0.0779 - accuracy: 0.9967 - val_loss: 1.3098 - val_accuracy: 0.5588


In [26]:
loss, accuracy = race_model.evaluate(X_test, y_test_race_encoded)
print(f"Test Accuracy: {accuracy}")

5/5 [==============================] - 1s 139ms/step - loss: 1.3098 - accuracy: 0.5588
Test Accuracy: 0.5588235259056091


In [25]:
train_df

,filename,gender,yob,race,image_path
678,00738_941201_fa_converted_final.png,Male,1954,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
448,00500_940519_fa_converted_final.png,Female,1944,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
673,00733_941201_fa_converted_final.png,Male,1954,Asian-Middle-Eastern,/home/mahdi/Projects/FairnessLens/FERET/colorf...
18,00053_931230_fa_converted_final.png,Female,1963,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
478,00530_940519_fa_converted_final.png,Female,1954,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
...,...,...,...,...,...
71,00107_941121_fa_converted_final.png,Female,1953,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
106,00143_931230_fa_a_converted_final.png,Male,1963,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
270,00315_940422_fa_converted_final.png,Female,1974,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
435,00487_940519_fa_converted_final.png,Female,1954,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...


In [26]:
test_df

,filename,gender,yob,race,image_path
647,00707_941205_fa_converted_final.png,Male,1954,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
607,00663_941121_fa_converted_final.png,Female,1974,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
63,00099_931230_fa_a_converted_final.png,Male,1963,Asian,/home/mahdi/Projects/FairnessLens/FERET/colorf...
319,00364_940422_fa_converted_final.png,Male,1974,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
101,00138_931230_fa_converted_final.png,Male,1973,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
...,...,...,...,...,...
110,00147_931230_fa_converted_final.png,Female,1963,Asian,/home/mahdi/Projects/FairnessLens/FERET/colorf...
82,00118_931230_fa_a_converted_final.png,Male,1953,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
51,00086_931230_fa_converted_final.png,Male,1963,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
428,00480_940519_fa_converted_final.png,Female,1954,White,/home/mahdi/Projects/FairnessLens/FERET/colorf...
